# **Project-Group-15**
Abhishek Sharma, Owen Pannucci, Krish Arulalan

Department of Computer Science, Rice University

as537@rice.edu, op21@rice.edu, ka72@rice.edu

NOTE: This code was executed on Kaggle with 2 x T4 GPUs.

# **Installing Required Libraries**

In [ ]:
!pip install transformers==4.40.2
!pip install bitsandbytes
!pip install -U triton==3.2.0
!pip install python-Levenshtein
import os
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor
import transformers
from transformers import pipeline, logging
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoProcessor

# Set environment variable to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Ensure both GPUs are visible
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 76.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 89.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 22.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━

# **Synthtic Nutrition Label Dataset Generation**

Downloading Fonts

In [2]:
!wget -O arial.ttf "https://raw.githubusercontent.com/gasharper/linux-fonts/master/arial.ttf"
!wget "https://raw.githubusercontent.com/RedBalloonShenanigans/MonitorDarkly/master/fonts/arialbd.ttf"

--2025-04-20 18:28:38--  https://raw.githubusercontent.com/gasharper/linux-fonts/master/arial.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 916684 (895K) [application/octet-stream]
Saving to: ‘arial.ttf’

arial.ttf           100%[===================>] 895.20K  --.-KB/s    in 0.05s   

2025-04-20 18:28:38 (19.3 MB/s) - ‘arial.ttf’ saved [916684/916684]

--2025-04-20 18:28:38--  https://raw.githubusercontent.com/RedBalloonShenanigans/MonitorDarkly/master/fonts/arialbd.ttf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 352224 

Code to Generate the Synthetic Dataset

In [3]:
from PIL import Image, ImageDraw, ImageFont
import json
import os
import random
import textwrap

NUMBER_OF_IMAGES = 500

# Create a directory to save the images and annotations
output_dir = "nutrition_labels"
images_dir = os.path.join(output_dir, "images")  # Subdirectory for images
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

# Define font paths (adjust these paths based on your system)
try:
    font_regular = ImageFont.truetype("arial.ttf", 16)  # For headers
    font_small = ImageFont.truetype("arial.ttf", 12)    # For regular text
    font_smaller = ImageFont.truetype("arial.ttf", 10)  # For ingredients
    font_bold = ImageFont.truetype("arialbd.ttf", 14)   # For calories and product name
except IOError:
    print("Font files not found. Using default font (results may vary).")
    font_regular = ImageFont.load_default()
    font_small = ImageFont.load_default()
    font_smaller = ImageFont.load_default()
    font_bold = ImageFont.load_default()

# Lists for random generation
flours = ["Whole Wheat Flour", "Enriched Flour (Wheat Flour, Niacin, Reduced Iron, Thiamine Mononitrate, Riboflavin, Folic Acid)", "Oat Flour", "Rice Flour"]  # Wheat already included
oils = ["Palm Oil", "Canola Oil", "Soybean Oil", "Coconut Oil", "Palm Kernel Oil", "Peanut Oil"]
sweeteners = ["Sugar", "Brown Sugar", "High Fructose Corn Syrup", "Honey", "Molasses", "Corn Syrup", "Corn Syrup Solids"]
additives = [
    "Salt", "Baking Soda", "Baking Powder", "Soy Lecithin", "Mono- and Diglycerides", 
    "Citric Acid", "Sodium Benzoate (Preservative)", "BHT (Preservative)", 
    "Mixed Tocopherols (to maintain freshness)", "Peanut Butter (Peanuts, Salt)", 
    "Whey Powder (Milk)", "Soy Protein", "Dried Egg Whites"  # Added milk, soy, egg allergens
]
nuts = ["Peanuts", "Almonds", "Walnuts", "Cashews"]  # Expanded tree nuts
flavors = ["Natural Vanilla Flavor", "Natural Caramel Flavor", "Natural Flavor", "Artificial Flavor", "Cinnamon"]
colors = ["Blue 1", "Blue 2", "Yellow 5", "Yellow 6", "Red 40", "Red 3", "Yellow 5 Lake", "Annatto Extract (Color)"]
product_names = ["CRUNCHY SNACKS", "SWEET BISCUITS", "OAT CRACKERS", "CHOCO BITES", "CARAMEL PUFFS", "SALTY CRACKERS", "VANILLA WAFERS", "HONEY BITES"]
background_colors = ["#FFFFFF", "#F5F5DC"]  # White, Off-white
text_colors = ["#000000", "#4A3728"]        # Black, Dark brown
product_name_colors = ["#000000", "#5C4033", "#C68E17"]  # Black, Chocolate brown, Caramel

# Function to generate random nutrition label text
def generate_nutrition_label_text(image_number):
    # Randomly decide if the table has two columns
    use_two_columns = random.choice([True, False])

    # Single serving size for the entire table
    pieces = random.randint(1, 3)
    grams = random.randint(20, 40)
    servings = random.randint(6, 15)
    calories = random.randint(80, 150)

    # Total Fat and its components
    total_fat = random.randint(4, 9)  # Total Fat between 4g and 9g
    min_sat_trans_sum = total_fat // 2
    saturated_fat = random.randint(1, min(3, total_fat // 2))  # Saturated Fat between 1g and 3g
    remaining_fat = total_fat - saturated_fat
    trans_fat = random.randint(0, min(1, max(1, min_sat_trans_sum - saturated_fat)))  # Trans Fat to ensure a significant portion
    other_fats = total_fat - (saturated_fat + trans_fat)  # Remaining fats (not displayed)

    cholesterol = random.randint(0, 10)
    sodium = random.randint(60, 120)

    # Total Carbohydrate and its components
    total_carbs = random.randint(9, 16)
    min_fiber_sugar_sum = total_carbs // 2
    dietary_fiber = random.randint(0, min(3, total_carbs // 3))  # Dietary Fiber between 0g and 3g
    remaining_carbs = total_carbs - dietary_fiber
    total_sugars = random.randint(1, min(remaining_carbs, max(5, min_fiber_sugar_sum - dietary_fiber)))  # Sugars to ensure a significant portion
    other_carbs = total_carbs - (dietary_fiber + total_sugars)  # Remaining carbs (not displayed)

    protein = random.randint(0, 3)

    # Micronutrients (to be displayed in a separate table below)
    vitamin_d = random.randint(0, 3)  # Always included
    calcium = random.randint(1, 3)
    iron = random.randint(2, 5)
    potassium = random.randint(1, 3)
    include_vitamin_a = random.choice([True, False])
    include_vitamin_c = random.choice([True, False])
    vitamin_a = random.randint(0, 5) if include_vitamin_a else None
    vitamin_c = random.randint(0, 5) if include_vitamin_c else None

    # Generate ingredients list
    ingredients = [
        random.choice(flours),
        random.choice(sweeteners),
        random.choice(oils),
        random.choice(nuts),
        random.choice(sweeteners),
        random.choice(additives),
        random.choice(additives),
        random.choice(additives),
        random.choice(flavors),
        random.choice(colors),
        random.choice(colors),
        random.choice(additives)
    ]
    ingredients = list(dict.fromkeys(ingredients))  # Remove duplicates
    ingredients_str = ", ".join(ingredients) + "."

    # Generate the text for the nutritional facts (main table entries)
    table_lines = [
        f"Calories {calories}",
        f"Total Fat {total_fat}g {int(total_fat/0.65)}%",
        f"Saturated Fat {saturated_fat}g {saturated_fat*5}%",
        f"Trans Fat {trans_fat}g",
        f"Cholesterol {cholesterol}mg {int(cholesterol/3)}%",
        f"Sodium {sodium}mg {int(sodium/240)}%",
        f"Total Carbohydrate {total_carbs}g {int(total_carbs/3)}%",
        f"Dietary Fiber {dietary_fiber}g {dietary_fiber*4}%",
        f"Sugars {total_sugars}g {total_sugars*2}%",
        f"Protein {protein}g"
    ]

    # Generate the text for micronutrients (second table)
    micronutrient_lines = [
        f"Vitamin D {vitamin_d}%"
    ]
    if include_vitamin_a:
        micronutrient_lines.append(f"Vitamin A {vitamin_a}%")
    if include_vitamin_c:
        micronutrient_lines.append(f"Vitamin C {vitamin_c}%")
    micronutrient_lines.extend([
        f"Calcium {calcium}%",
        f"Iron {iron}%",
        f"Potassium {potassium}%"
    ])

    # Generate product code, name, and number
    product_code = f"RB{random.randint(10000, 99999)}"
    product_name = random.choice(product_names)
    product_number = str(random.randint(100, 999))

    # Combine all text lines, including product code, name, and number with newlines
    text_lines = [
        "Nutrition Facts",
        f"Serving Size {pieces} piece{'s' if pieces > 1 else ''} ({grams}g)",
        f"Servings Per Container about {servings}"
    ]
    text_lines.extend(table_lines)
    text_lines.extend(micronutrient_lines)
    text_lines.append(f"Ingredients: {ingredients_str}")
    # Add product code, name, and number as separate lines
    text_lines.append(product_code)
    text_lines.append(product_name)
    text_lines.append(product_number)
    text = "\n".join(text_lines)

    # Add the headings to the text for annotations only
    lines = text.split('\n')
    annotation_text_lines = lines[:3]  # Up to "Servings Per Container"
    if use_two_columns:
        annotation_text_lines.append("Amount Per Serving\n%Daily Value*\nAmount Per Serving\n%Daily Value*")
    else:
        annotation_text_lines.append("Amount Per Serving\n%Daily Value*")
    annotation_text_lines.extend(lines[3:])  # Add the rest of the lines
    annotation_text = "\n".join(annotation_text_lines)

    return {
        "filename": f"nutrition_label_{image_number:03d}.png",
        "text": text,  # Original text without headings, used for drawing
        "annotation_text": annotation_text,  # Text with headings, used for annotations
        "background_color": random.choice(background_colors),
        "text_color": random.choice(text_colors),
        "product_name_color": random.choice(product_name_colors),
        "ingredients": ingredients_str,
        "use_two_columns": use_two_columns,
        "table_lines": table_lines,
        "micronutrient_lines": micronutrient_lines
    }

# Function to wrap text to fit within a specified width
def wrap_text(draw, text, font, max_width):
    lines = []
    words = text.split(', ')
    current_line = words[0]
    for word in words[1:]:
        test_line = current_line + ', ' + word
        width = draw.textlength(test_line, font=font)
        if width <= max_width:
            current_line = test_line
        else:
            lines.append(current_line + ',')  # Add a comma at the end of wrapped lines
            current_line = word
    lines.append(current_line)  # Final line ends with a period
    return lines

# Function to generate an image from the text
def generate_image(data):
    # Set image dimensions based on whether it's one or two columns
    use_two_columns = data["use_two_columns"]
    if use_two_columns:
        img_width, img_height = 600, 600  # Increased height for two columns
    else:
        img_width, img_height = 600, 700  # Taller than wide for one column

    img = Image.new('RGB', (img_width, img_height), color=data["background_color"])
    draw = ImageDraw.Draw(img)

    lines = data["text"].split('\n')  # Use the original text without headings
    y_position = 20
    table_start_y = 0
    table_end_y = 0
    in_table = False
    table_entries = []
    current_column = 1  # Start with the first column
    col1_y_position = 0  # Track y_position for column 1
    col2_y_position = 0  # Track y_position for column 2

    # First pass: Collect table entries and draw the main table
    for line in lines:
        if line.startswith("Nutrition Facts"):
            draw.text((20, y_position), line, font=font_regular, fill=data["text_color"])
            y_position += 30
            draw.line((20, y_position, img_width - 20, y_position), fill=data["text_color"], width=1)
            y_position += 10
        elif line.startswith("Serving Size"):
            draw.text((20, y_position), line, font=font_small, fill=data["text_color"])
            y_position += 20
        elif line.startswith("Servings Per Container"):
            draw.text((20, y_position), line, font=font_small, fill=data["text_color"])
            y_position += 20
            # Draw table headings with a larger gap
            if use_two_columns:
                draw.text((25, y_position), "Amount Per Serving", font=font_small, fill=data["text_color"])
                draw.text((170, y_position), "%Daily Value*", font=font_small, fill=data["text_color"])
                draw.text((315, y_position), "Amount Per Serving", font=font_small, fill=data["text_color"])
                draw.text((460, y_position), "%Daily Value*", font=font_small, fill=data["text_color"])
            else:
                # Calculate the width of "%Daily Value*" to position it properly
                daily_value_text = "%Daily Value*"
                daily_value_width = draw.textlength(daily_value_text, font=font_small)
                draw.text((25, y_position), "Amount Per Serving", font=font_small, fill=data["text_color"])
                draw.text((img_width - daily_value_width - 20, y_position), daily_value_text, font=font_small, fill=data["text_color"])
            y_position += 20
            draw.line((20, y_position, img_width - 20, y_position), fill=data["text_color"], width=1)
            y_position += 5
            table_start_y = y_position
            in_table = True
        elif line.startswith("Vitamin") or line in [f"Calcium {data['text'].split('Calcium ')[1].split('%')[0]}%", f"Iron {data['text'].split('Iron ')[1].split('%')[0]}%", f"Potassium {data['text'].split('Potassium ')[1].split('%')[0]}%"]:
            in_table = False
            # These will be drawn in a second table
            continue
        elif line.startswith("Ingredients:"):
            in_table = False
            # table_end_y will be set after drawing the table entries
        else:
            if in_table:
                table_entries.append(line)

    # Second pass: Draw the main table entries, splitting across columns if needed
    if table_entries:
        split_point = len(table_entries) // 2  # Split the entries roughly in half
        col1_y_position = table_start_y
        col2_y_position = table_start_y
        for i, entry in enumerate(table_entries):
            if use_two_columns and i == split_point:
                current_column = 2
            # Determine if the entry should be indented
            indent = 0
            if entry.startswith("Saturated Fat") or entry.startswith("Trans Fat") or entry.startswith("Dietary Fiber") or entry.startswith("Sugars"):
                indent = 10  # Indent by 10 pixels

            if current_column == 1:
                if '%' in entry:
                    parts = entry.rsplit(' ', 1)
                    nutrient = parts[0]
                    percentage = parts[1]
                    if use_two_columns:
                        draw.text((25 + indent, col1_y_position), nutrient, font=font_small, fill=data["text_color"])
                        draw.text((170, col1_y_position), percentage, font=font_small, fill=data["text_color"])
                    else:
                        draw.text((25 + indent, col1_y_position), nutrient, font=font_small, fill=data["text_color"])
                        draw.text((img_width - 50, col1_y_position), percentage, font=font_small, fill=data["text_color"])
                else:
                    if use_two_columns:
                        draw.text((25 + indent, col1_y_position), entry, font=font_small, fill=data["text_color"])
                    else:
                        draw.text((25 + indent, col1_y_position), entry, font=font_small, fill=data["text_color"])
                col1_y_position += 20
            elif current_column == 2:
                if '%' in entry:
                    parts = entry.rsplit(' ', 1)
                    nutrient = parts[0]
                    percentage = parts[1]
                    draw.text((315 + indent, col2_y_position), nutrient, font=font_small, fill=data["text_color"])
                    draw.text((460, col2_y_position), percentage, font=font_small, fill=data["text_color"])
                else:
                    draw.text((315 + indent, col2_y_position), entry, font=font_small, fill=data["text_color"])
                col2_y_position += 20

        # Determine the end of the main table based on the tallest column
        table_end_y = max(col1_y_position, col2_y_position if use_two_columns else 0)

        # Draw the main table borders
        draw.rectangle(
            (20, table_start_y - 5, img_width - 20, table_end_y - 5),
            outline=data["text_color"],
            width=1
        )
        # Draw vertical line for two-column layout
        if use_two_columns:
            draw.line(
                (300, table_start_y - 25, 300, table_end_y - 5),
                fill=data["text_color"],
                width=1
            )
        # Draw horizontal lines for each table row
        max_rows = len(table_entries) if not use_two_columns else max(len([e for i, e in enumerate(table_entries) if i < split_point]), len([e for i, e in enumerate(table_entries) if i >= split_point]))
        for i in range(max_rows + 1):
            row_y = table_start_y + (i * 20)
            draw.line(
                (20, row_y - 5, img_width - 20, row_y - 5),
                fill=data["text_color"],
                width=1
            )

    # Third pass: Draw the second table for micronutrients
    y_position = table_end_y + 10  # Start below the main table
    micronutrient_start_y = y_position
    for micronutrient in data["micronutrient_lines"]:
        parts = micronutrient.rsplit(' ', 1)
        nutrient = parts[0]
        percentage = parts[1]
        if use_two_columns:
            # In two-column layout, the second table is a single column with percentages on the far right
            draw.text((25, y_position), nutrient, font=font_small, fill=data["text_color"])
            draw.text((460, y_position), percentage, font=font_small, fill=data["text_color"])
        else:
            # In one-column layout, align with the main table's percentages
            draw.text((25, y_position), nutrient, font=font_small, fill=data["text_color"])
            draw.text((img_width - 50, y_position), percentage, font=font_small, fill=data["text_color"])
        y_position += 20

    # Draw the second table borders
    micronutrient_end_y = y_position
    draw.rectangle(
        (20, micronutrient_start_y - 5, img_width - 20, micronutrient_end_y - 5),
        outline=data["text_color"],
        width=1
    )
    # Draw horizontal lines for each row in the second table
    for i in range(len(data["micronutrient_lines"]) + 1):
        row_y = micronutrient_start_y + (i * 20)
        draw.line(
            (20, row_y - 5, img_width - 20, row_y - 5),
            fill=data["text_color"],
            width=1
        )

    # Fourth pass: Draw the ingredients and product code below the second table
    y_position = micronutrient_end_y + 10  # Start below the second table
    for line in lines:
        if line.startswith("Ingredients:"):
            # Add the "Ingredients:" text directly with the wrapped ingredients
            max_width = img_width - 40  # Width of the image minus margins
            ingredients_text = f"Ingredients: {data['ingredients']}"
            wrapped_lines = wrap_text(draw, ingredients_text, font_small, max_width)
            # Draw the wrapped ingredients text
            for wrapped_line in wrapped_lines:
                draw.text((20, y_position), wrapped_line, font=font_small, fill=data["text_color"])
                y_position += 20

    # Extract product code, name, and number from the last three lines of text
    product_code = lines[-3]  # Third-to-last line
    product_name = lines[-2]  # Second-to-last line
    product_number = lines[-1]  # Last line

    # Draw the product code, name, and number on the same line with gaps
    gap = 40  # Small gap between elements in pixels
    x_position = 20  # Starting position
    # Draw the code
    draw.text((x_position, y_position), product_code, font=font_small, fill=data["text_color"])
    code_width = draw.textlength(product_code, font=font_small)
    x_position += code_width + gap  # Move to the end of the code plus a gap
    # Draw the product name
    draw.text((x_position, y_position), product_name, font=font_small, fill=data["product_name_color"])
    product_name_width = draw.textlength(product_name, font=font_small)
    x_position += product_name_width + gap  # Move to the end of the product name plus a gap
    # Draw the number
    draw.text((x_position, y_position), product_number, font=font_small, fill=data["text_color"])
    y_position += 30

    # Save the image in the images subdirectory
    img_path = os.path.join(images_dir, data["filename"])
    img.save(img_path)
    print(f"Saved image: {img_path}")
    return data["filename"], data["annotation_text"]

# Main function to generate a specified number of images
def generate_nutrition_labels(num_images):
    annotations = {}
    for i in range(1, num_images + 1):
        data = generate_nutrition_label_text(i)
        filename, annotation_text = generate_image(data)
        annotations[filename] = {
            "text": annotation_text
        }

    # Save annotations directly in the output_dir (not in the images subdirectory)
    annotations_path = os.path.join(output_dir, "annotations.json")
    with open(annotations_path, "w") as f:
        json.dump(annotations, f, indent=2)
    print(f"Saved annotations: {annotations_path}")

# Generate 5 images (you can change this number)
num_images = NUMBER_OF_IMAGES
generate_nutrition_labels(num_images)

Saved image: nutrition_labels/images/nutrition_label_001.png
Saved image: nutrition_labels/images/nutrition_label_002.png
Saved image: nutrition_labels/images/nutrition_label_003.png
Saved image: nutrition_labels/images/nutrition_label_004.png
Saved image: nutrition_labels/images/nutrition_label_005.png
Saved image: nutrition_labels/images/nutrition_label_006.png
Saved image: nutrition_labels/images/nutrition_label_007.png
Saved image: nutrition_labels/images/nutrition_label_008.png
Saved image: nutrition_labels/images/nutrition_label_009.png
Saved image: nutrition_labels/images/nutrition_label_010.png
Saved image: nutrition_labels/images/nutrition_label_011.png
Saved image: nutrition_labels/images/nutrition_label_012.png
Saved image: nutrition_labels/images/nutrition_label_013.png
Saved image: nutrition_labels/images/nutrition_label_014.png
Saved image: nutrition_labels/images/nutrition_label_015.png
Saved image: nutrition_labels/images/nutrition_label_016.png
Saved image: nutrition_l

# **OCR Model Setup**

Loading in the OCR model

In [4]:
ocr_model_id = "microsoft/Phi-3-vision-128k-instruct"

ocr_model = AutoModelForCausalLM.from_pretrained(ocr_model_id, device_map={"": 0}, trust_remote_code=True, torch_dtype="auto", _attn_implementation='eager') # use _attn_implementation='eager' to disable flash attention

ocr_processor = AutoProcessor.from_pretrained(ocr_model_id, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3_v.py:   0%|          | 0.00/75.4k [00:00<?, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-vision-128k-instruct:
- processing_phi3_v.py
- image_processing_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Post-Processing Function for OCR Model

In [5]:
def postProcessOCR(prediction):
    lines = prediction.split('\n')
    processed_lines = []
    i = 0
    in_ingredients = False
    ingredients_lines = []

    while i < len(lines):
        line = lines[i].strip()

        # Handle ingredients section
        if line.startswith("Ingredients:"):
            in_ingredients = True
            ingredients_lines.append(line)
            i += 1
            continue
        elif in_ingredients:
            # Check if the current line ends with a period
            if line.strip().endswith('.'):
                # Include the current line (ending with a period) in the ingredients
                ingredients_lines.append(line)
                in_ingredients = False
                # Clean up ingredients: remove trailing commas, and join into a single line
                cleaned_ingredients = []
                for ing_line in ingredients_lines:
                    cleaned_line = ing_line.strip().rstrip('')
                    if cleaned_line:  # Only add non-empty lines
                        cleaned_ingredients.append(cleaned_line)
                # Join all ingredients into a single line
                ingredients_str = " ".join(cleaned_ingredients)
                # Remove any double spaces
                ingredients_str = " ".join(ingredients_str.split())

                # Add the ingredients line
                processed_lines.append(ingredients_str)

                # Move to the next line after the period
                i += 1

                # Process the remaining lines (product info)
                remaining_lines = []
                while i < len(lines):
                    remaining_line = lines[i].strip()
                    if remaining_line:  # Only add non-empty lines
                        remaining_lines.append(remaining_line)
                    i += 1

                # If there are remaining lines, add them as-is
                if remaining_lines:
                    processed_lines.extend(remaining_lines)
                break  # Exit the loop since we've processed all remaining lines
            else:
                ingredients_lines.append(line)
                i += 1
                continue

        # Check if the current line is a percentage (e.g., "9%")
        if i + 1 < len(lines):
            percentage = lines[i + 1].strip()
            is_percentage = (percentage.endswith('%') and
                             percentage[:-1].replace('.', '', 1).replace(' ', '').replace(',', '').isdigit())
            if is_percentage:
                # Combine the current line with the percentage
                processed_lines.append(f"{line} {percentage}")
                i += 2  # Skip the percentage line
                continue

        # Add the line as is
        if line:  # Only add non-empty lines
            processed_lines.append(line)
        i += 1

    # If the last section was ingredients, join and append it
    if in_ingredients:
        cleaned_ingredients = []
        for ing_line in ingredients_lines:
            cleaned_line = ing_line.strip().rstrip(',')
            if cleaned_line:
                cleaned_ingredients.append(cleaned_line)
        ingredients_str = " ".join(cleaned_ingredients)
        ingredients_str = " ".join(ingredients_str.split())
        processed_lines.append(ingredients_str)

    # Remove any blank lines from the final output
    processed_lines = [line for line in processed_lines if line.strip()]

    return "\n".join(processed_lines)

Function to Run An Image In The OCR Model

In [6]:
def runOCR(imagePath):

  messages = [
      {"role": "user", "content": "<|image_1|>\nProvide OCR for all the text in given image. In the output just include the text in the image and have the relevant parts all on the same line."}
  ]

  image = Image.open(imagePath)

  ocr_prompt = ocr_processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  ocr_inputs = ocr_processor(ocr_prompt, [image], return_tensors="pt").to("cuda:0")

  ocr_generation_args = {
      "max_new_tokens": 1000,
      "temperature": 0.0,
      "do_sample": False,
  }

  ocr_generate_ids = ocr_model.generate(**ocr_inputs, eos_token_id=ocr_processor.tokenizer.eos_token_id, **ocr_generation_args)

  # remove input tokens
  ocr_generate_ids = ocr_generate_ids[:, ocr_inputs['input_ids'].shape[1]:]
  ocr_response = ocr_processor.batch_decode(ocr_generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

  # post-process the response
  ocr_post_processed_response = postProcessOCR(ocr_response)

  return ocr_post_processed_response

# **OCR Model Validation**

In [8]:
import os
import json
from PIL import Image
import torch
from Levenshtein import distance as levenshtein_distance  # For partial text comparison

# Define the validation function
def validation(images_dir, annotations_path):
    """
    Validates predicted text from images against true annotations by comparing lines directly, ignoring order.

    Args:
        images_dir (str): Path to the directory containing image files.
        annotations_path (str): Path to the annotations JSON file.

    Returns:
        dict: Validation results including accuracy and mismatches.
    """
    # Load annotations
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)

    # Get list of image files
    image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.png')])

    # Ensure the number of images matches the number of annotations
    if len(image_files) != len(annotations):
        raise ValueError(f"Mismatch: {len(image_files)} images but {len(annotations)} annotations")

    # Initialize counters for validation
    total_images = len(image_files)
    correct_predictions = 0
    mismatches = []

    # Process each image
    for i in range(total_images):
        # Get the image and its true label
        image_filename = image_files[i]
        image_path = os.path.join(images_dir, image_filename)
        if image_filename not in annotations:
            print(f"Warning: No annotation found for {image_filename}, skipping...")
            continue

        true_label = annotations[image_filename]["text"]

        prediction = runOCR(image_path)

        # Log warning if Ingredients section is missing
        if "Ingredients:" not in prediction:
            print(f"Warning: No Ingredients section found in prediction for image {image_filename}")

        # Split true label and prediction into lines, removing empty lines
        true_lines = [line.strip() for line in true_label.split('\n') if line.strip()]
        pred_lines = [line.strip() for line in prediction.split('\n') if line.strip()]

        # Compare lines as sets (order-independent)
        true_lines_set = set(line.lower() for line in true_lines)
        pred_lines_set = set(line.lower() for line in pred_lines)

        # Check if the sets match exactly
        is_correct = True
        mismatches_details = []

        if true_lines_set != pred_lines_set:
            is_correct = False
            # Identify missing and extra lines for detailed reporting
            missing_lines = true_lines_set - pred_lines_set  # Lines in true but not in pred
            extra_lines = pred_lines_set - true_lines_set    # Lines in pred but not in true
            # Add separate entries for missing and extra lines
            if missing_lines:
                mismatches_details.append(f"Missing lines (in true but not in pred):\n{list(missing_lines)}")
            if extra_lines:
                mismatches_details.append(f"Extra lines (in pred but not in true):\n{list(extra_lines)}")

        if is_correct:
            correct_predictions += 1
        else:
            lev_distance = levenshtein_distance(true_label.lower(), prediction.lower())
            mismatches.append({
                "image": image_filename,
                "true_label": true_label,
                "prediction": prediction,
                "levenshtein_distance": lev_distance,
                "details": mismatches_details
            })

        print(f"Processed {i+1}/{total_images}: {image_filename} - {'Correct' if is_correct else 'Incorrect'}")

    # Compute accuracy
    accuracy = correct_predictions / total_images if total_images > 0 else 0

    # Return validation results
    return {
        "total_images": total_images,
        "correct_predictions": correct_predictions,
        "accuracy": accuracy,
        "mismatches": mismatches
    }

# Setting up Validation
dataset_dir = "nutrition_labels"
images_dir = os.path.join(dataset_dir, "images")
annotations_path = os.path.join(dataset_dir, "annotations.json")

# Run validation
print("Starting Validation...")
validation_results = validation(images_dir, annotations_path)
print("\nValidation Results:")
print(f"Total Images: {validation_results['total_images']}")
print(f"Correct Predictions: {validation_results['correct_predictions']}")
print(f"Accuracy: {validation_results['accuracy']:.2%}")
if validation_results['mismatches']:
    print("\nMismatches:\n")
    for mismatch in validation_results['mismatches']:
        print(f"Image: {mismatch['image']}\n")
        print(f"True Label:\n{mismatch['true_label']}\n")
        print(f"Prediction:\n{mismatch['prediction']}\n")
        print(f"Levenshtein Distance: {mismatch['levenshtein_distance']}\n")
        print("Details:")
        for detail in mismatch['details']:
            print(detail)
        print("-" * 50)
else:
    print("No mismatches found!")

Starting Validation...
Processed 1/500: nutrition_label_001.png - Correct
Processed 2/500: nutrition_label_002.png - Correct
Processed 3/500: nutrition_label_003.png - Correct
Processed 4/500: nutrition_label_004.png - Correct
Processed 5/500: nutrition_label_005.png - Correct
Processed 6/500: nutrition_label_006.png - Correct
Processed 7/500: nutrition_label_007.png - Correct
Processed 8/500: nutrition_label_008.png - Correct
Processed 9/500: nutrition_label_009.png - Correct
Processed 10/500: nutrition_label_010.png - Correct
Processed 11/500: nutrition_label_011.png - Correct
Processed 12/500: nutrition_label_012.png - Correct
Processed 13/500: nutrition_label_013.png - Correct
Processed 14/500: nutrition_label_014.png - Correct
Processed 15/500: nutrition_label_015.png - Correct
Processed 16/500: nutrition_label_016.png - Correct
Processed 17/500: nutrition_label_017.png - Correct
Processed 18/500: nutrition_label_018.png - Correct
Processed 19/500: nutrition_label_019.png - Correc

# **LLM Model Setup (meta-llama/Llama-2-7b-chat-hf)**

Loading in the LLM model

In [6]:
# Retrieve Hugging Face token
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("llama2-chat")
login(token=hf_token)

# LLM Model (LLaMA-2-7B-chat) - Place on GPU 1
llm_model_id = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(llm_model_id)
llm_model = AutoModelForCausalLM.from_pretrained(
    llm_model_id,
    torch_dtype=torch.bfloat16,
    device_map={"": 1},  # Explicitly place on GPU 1
)

# Initialize pipeline
llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=tokenizer,
    device_map={"": 1},  # Run pipeline on GPU 1
    torch_dtype=torch.bfloat16,
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Pre-Processing Function for LLM

In [7]:
def preProcessLLM(label_data, user_queries=None):
  nutrition_info = label_data

  default_queries = [
      "Provide a brief summary of the nutritional profile.",
      "Any health risks or red flags?",
      "Dietary suitability."
  ]

  if user_queries:
      default_queries.extend(user_queries)

  query_text = "\n".join(f"{i+1}. {q}" for i, q in enumerate(default_queries))

  user_message = f"""Here is the nutrition information:\n{nutrition_info}\nProvide:\n{query_text}"""
  return user_message

Function to Run The LLM Model

In [98]:
def runLLM(label_data, user_queries=None):
    user_message = preProcessLLM(label_data, user_queries)

    # Format prompt for LLaMA-2 chat model
    messages = [
        {
          "role": "system",
          "content": "You are a food, nutrition, and allergy expert. Your task is to analyze text extracted from a food nutrition label and answer user questions about nutritional content, ingredients, serving size, allergens, or other details present in the label. Base your answers solely on the provided text, without making assumptions or adding external knowledge.\n\nFor allergens: Only identify an allergen if it is explicitly listed in the ingredient list (e.g., 'milk', 'peanuts') or in a 'Contains' statement (e.g., 'Contains: wheat'). Do not assume an allergen is present based on vague terms like 'flour' or 'powder' unless the label specifies the source (e.g., 'wheat flour'). Do not assume cross-reactivity between allergens (e.g., do not equate a tree nut allergy with a peanut allergy unless explicitly stated).\n\nFor dietary suitability: Confirm suitability for specific diets (e.g., gluten-free, vegan) only if supported by the label. A product can be considered gluten-free if it lacks gluten-containing ingredients (e.g., wheat, barley, rye) in the ingredient list, but note potential cross-contamination risks unless explicitly labeled 'gluten-free'. For vegan suitability, ensure no animal-derived ingredients (e.g., milk, eggs) are listed. Do not speculate about suitability for unlisted dietary restrictions or processing-related contamination unless stated.\n\nProvide accurate, concise, and clear answers. If a question cannot be answered due to missing or ambiguous information, state so clearly and explain why. Avoid referencing unlisted ingredients, health risks, or nutritional concerns not explicitly mentioned in the provided text."
        },
        {"role": "user", "content": user_message}
    ]

    # Convert messages to a single prompt (LLaMA-2 chat format)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    # Run the pipeline
    outputs = llm_pipeline(
        prompt,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        return_full_text=False  # Only return the generated text, not the prompt
    )

    # Post-process to ensure new lines between question-answer pairs
    response = outputs[0]["generated_text"].strip()
    lines = response.split("\n")
    formatted_response = "\n\n".join(line.strip() for line in lines if line.strip())
    return formatted_response

# **Run Entire Pipeline (OCR + LLM (Llama-2-7b-chat-hf))**

In [130]:
# path to nutrition label
nutrition_label = "/kaggle/input/nutritionlabel1/real_nutrition_label_010.png"

image = Image.open(nutrition_label)

# user queries:
user_queries = [
    "Are any ingredients genetically modified (GMO)?",
    "Are there emulsifiers or stabilizers (like soy lecithin, carrageenan)?"
]

def run(nutrition_label, user_queries):
  
  OCR = runOCR(nutrition_label)
  LLM = runLLM(OCR, user_queries)

  print(f"OCR Output:\n\n{OCR}\n")
  print(f"LLM Output:\n\n{LLM}\n")

run(nutrition_label, user_queries)

OCR Output:

Nutrition Facts
Serving Size
2 Full Cracker Sheets (31g)
Servings Per Container about 13
Amount Per Serving
Calories 130
Calories from Fat 25
% Daily Value*
Total Fat 3g 5%
Saturated Fat 0.5g 3%
Trans Fat 0g
Cholesterol 0mg 0%
Sodium 135mg 6%
Total Carbohydrate 24g 8%
Dietary Fiber 1g 4%
Sugars 8g
Protein 2g
Vitamin A 0%
Vitamin C 0%
Calcium 0%
Iron 6%
*Percent Daily Values are based on a 2,000 calorie diet. Your daily values may be higher
or lower depending on your calorie needs.
Calories:
2,000
2,500
Total Fat
Less than 65g
80g
Saturated Fat
Less than 20g
25g
Cholesterol
Less than 300mg
300mg
Sodium
Less than 2,400mg 2,400mg
Total Carbohydrate
300g
375g
Dietary Fiber
25g
30g
INGREDIENTS: ENRICHED FLOUR (WHEAT FLOUR,
NIACIN, IRON, THIAMINE MONONITRATE, RIBOFLAVIN,
FOLIC ACID), GRAHAM FLOUR (WHOLE GRAIN WHEAT
FLOUR), SUGAR, CANOLA AND/OR SOYBEAN AND/OR
PALM OIL, HONEY, LEAVENING (BAKING SODA,
CALCIUM PHOSPHATE), SALT, NATURAL FLAVOR,
SOY LECITHIN, SODIUM SULFITE.
CONTAINS: